In [5]:
from selenium import webdriver
from selenium.webdriver.common import action_chains, keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import random
import re

#setup our webdriver with a chrome profile and navigate to the docket portal
driver = webdriver.Chrome("C:/Users/KNguyen/chromedriver.exe")
driver.get('http://programs.dsireusa.org/system/program')
time.sleep(1)
#Apply Filter
driver.find_element_by_css_selector("#DataTables_Table_0_wrapper > button").click()
time.sleep(1)
#Technology
driver.find_element_by_css_selector("#filtersFlyout > form > nav > div > ul > li:nth-child(2) > a").click()
time.sleep(1)
#Renewable Energy
driver.find_element_by_css_selector("#filtersFlyout > form > nav > div > ul > li:nth-child(2) > div > ul > li:nth-child(3) > a").click()
time.sleep(1)
#Solar Technology
driver.find_element_by_css_selector("#filtersFlyout > form > nav > div > ul > li:nth-child(2) > div > ul > li:nth-child(3) > div > ul > li:nth-child(9) > a").click()
time.sleep(1)
#All Solar Technologies
driver.find_element_by_css_selector("#filtersFlyout > form > nav > div > ul > li:nth-child(2) > div > ul > li:nth-child(3) > div > ul > li:nth-child(9) > div > ul > li:nth-child(3) > label").click()
#time.sleep(10)
driver.implicitly_wait(10)
#Apply Filter button
driver.find_element_by_css_selector("#filtersFlyout > form > nav > div > ul > li:nth-child(2) > div > ul > li:nth-child(3) > div > ul > li:nth-child(9) > div > ul > li:nth-child(14) > button").click()
#try:
    #element = WebDriverWait(driver, 10).until(
        #EC.element_to_be_clickable((By.CLASS_NAME, "apply-filters-btn")))
#finally:
    #element.click()
    #driver.find_element_by_class_name("apply-filters-btn").click()
    
#link = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "apply-filters-btn")))
#link.click()
time.sleep(3)
#Select 'All'
driver.find_element_by_css_selector("#DataTables_Table_0_length > label > select > option[value='-1']").click()

In [11]:
sum_blob = driver.find_element_by_css_selector("body > div.content-container > main > div:nth-child(2) > div.status-bar > div > p").text
match_prog = re.search(r'\d+',sum_blob)
if match_prog:
    print ("Number of programs: " + match_prog.group())
else:
    print ('not found')

Number of programs: 1117


In [14]:
results_dirty = driver.find_element_by_css_selector("#DataTables_Table_0 > tbody")
results_html = results_dirty.get_attribute("innerHTML")
results_segs = results_html.split("<tr")
#print (results_segs[1])

 role="row" class="odd"><td><a href="/system/program/detail/1151" title="View program">NJ Clean Energy- WARMAdvantage Program (Electric and Gas)</a></td><td>NJ</td><td>Financial Incentive</td><td>Rebate Program</td><td>06/16/2005</td><td class="sorting_1">12/02/2016</td></tr>


In [36]:
with open('C:/Users/KNguyen/DSIRE.csv', 'w', newline='') as ncuc:
    a = csv.writer(ncuc, delimiter=',')
    bill_rows = [['Name', 'Link', 'State', 'Category','Policy/Incentive Type','Created', 'Last Updated']]

    for item in results_segs[1:]:
        match_link = re.search(r'(href=")(.*\d+)(" title)',item)
        if match_link:
            link = "http://programs.dsireusa.org" + match_link.group(2)
        else:
            link = 'not available'
            #print (link)

        match_program = re.search(r'(View program">)(.*)(</a>)', item)
        if match_program:
            name = match_program.group(2)
        else:
            name = 'not available'
            #print (name)

        match_state = re.search(r'(<td>)([A-Z]{2})(</td>)',item)
        if match_state:
            state = match_state.group(2)
        else:
            state = 'not available'
            #print (state)

        one_clean = item.replace("</td><td","</td><td>").split("</td><td>")
        category = one_clean[-4].replace('>','')
        #print (category)

        pi_type = one_clean[-3].replace('>','')
        #print (pi_type)

        created_date = one_clean[-2].replace('>','')
        #print (created_date)

        match_updated = re.search(r'\d+/\d+/\d+',one_clean[-1])
        if match_updated:
            updated_date = match_updated.group()
        else:
            updated_date = 'not available'
            #print (updated_date)
        #update = one_clean[-1]
        #print (update)
        
        newrow = [name, link, state, category, pi_type, created_date, updated_date]
        bill_rows.append(newrow)
        
    a.writerows(bill_rows)